# IBM APPLIED DATA SCIENCE CAPSTONE PROJECT - COURSERA

## Brewery Placement

### Import Dependencies

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geocoder
import geocoder
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML

In [2]:
from IPython.display import display_html
import pandas as pd
import numpy as np
import json 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library
from bs4 import BeautifulSoup as bs
import urllib.request
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Pull Denver CSV Data Into DataFrame

In [3]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Population,Low Income,Medium Income,High Income
0,Alder-Three Sisters Park,381,544,614,452
1,Allendale,1718,1672,1158,457
2,Alta Vista and Memorial Park,1417,1393,1015,404
3,"Apel Bacher Park, Koch Sub and Coulehan Grange",467,600,359,65
4,"Applewood, Echo Hill, Rolling Hills and Meadows",433,456,549,531


### Change Name

In [4]:
den_df=df_data_0
den_df.head()
print('Data Types: ',den_df.dtypes)

Data Types:  Neighborhood     object
Population        int64
Low Income        int64
Medium Income     int64
High Income       int64
dtype: object


### Check shape of dataframe

In [5]:
den_df.shape

(262, 5)

### Use geocoder to allocate latitude and longitude data

In [6]:
#define a function to get coordinates by neighborhood
def get_coord(neighbhorhood):
    #initialize none type variable
    coords=None
    #loop function to acquire coordinates
    while(coords is None):
        g=geocoder.arcgis('{}, Denver, Colorado'.format(neighbhorhood))
        coords=g.latlng
    return coords

In [7]:
den_coords = [ get_coord(neighborhood) for neighborhood in den_df['Neighborhood'].tolist()]
den_coords

[[39.74001000000004, -104.99201999999997],
 [39.693065281840674, -104.94054194098159],
 [37.171692, -104.5202805],
 [39.74001000000004, -104.99201999999997],
 [39.74001000000004, -104.99201999999997],
 [39.722661000000016, -105.11099945119341],
 [39.70396000000005, -105.01038999999997],
 [39.745750000000044, -105.00996999999995],
 [39.684840000000065, -104.78208999999998],
 [39.71117000000004, -104.99208999999996],
 [39.71815000000004, -105.03308999999996],
 [39.71815000000004, -105.04509999999999],
 [39.94160000000005, -104.73345999999998],
 [39.66172000000006, -105.06560999999999],
 [39.70384000000007, -104.94973999999996],
 [39.87560471069216, -104.9912022042408],
 [39.97957000000008, -104.80109999999996],
 [39.77666000000005, -105.03983999999997],
 [39.55499000000003, -105.29220999999995],
 [39.92853000000008, -105.00868999999994],
 [40.036830000000066, -105.23285999999996],
 [39.616930000000025, -105.02803999999998],
 [39.66386849999999, -104.94477900000001],
 [39.71532189738278, 

### Convert coordinates to dataframe and merge with Denver data

In [8]:
den_coordsdf=pd.DataFrame(den_coords, columns=['Latitude','Longitude'])

In [9]:
den_df['Latitude']=den_coordsdf['Latitude']
den_df['Longitude']=den_coordsdf['Longitude']
den_df.head()

,Neighborhood,Population,Low Income,Medium Income,High Income,Latitude,Longitude
0,Alder-Three Sisters Park,381,544,614,452,39.740010,-104.992020
1,Allendale,1718,1672,1158,457,39.693065,-104.940542
2,Alta Vista and Memorial Park,1417,1393,1015,404,37.171692,-104.520280
3,"Apel Bacher Park, Koch Sub and Coulehan Grange",467,600,359,65,39.740010,-104.992020
4,"Applewood, Echo Hill, Rolling Hills and Meadows",433,456,549,531,39.740010,-104.992020


### Backup new data from to cloud storage

In [10]:
den_df.to_csv('den_df.csv',index=False)

### Map Denver Neighborhoods

In [11]:
###Find Denver Coordinates
address = 'Denver, Colorado'
geolocator = Nominatim(user_agent='denver_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geospatial coordinates of Denver, CO are {}, {}'.format(latitude, longitude))

The geospatial coordinates of Denver, CO are 39.7392364, -104.9848623


In [12]:
### Create the Denver Neighborhood Map
map_den = folium.Map(location=[latitude, longitude],zoom_start=11)

for lat,lng,neighborhood in zip(den_df['Latitude'],den_df['Longitude'],den_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_den)
map_den

### Save Image File

In [13]:
map_den.save('map_den.html')

In [14]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: Z3UHF2NY3JYZ0SEUZVI33QVDOCDTKDILW2ZJIVWDU4I01JSJ
CLIENT_SECRET:XDDSKVRVRA3QR53MYUYBBQMTTMXEYZF5FZTVHN5KOCXL2EGR


### Get venue information from FourSquare Places API

In [15]:
search_query = 'brewery'

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            search_query,
            radius, 
            LIMIT)           
           
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])            

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #nearby_venues.columns = ['Neighborhood', 
                  #'Neighborhood Latitude', 
                  #'Neighborhood Longitude', 
                  #'Venue', 
                  #'Venue Latitude', 
                  #'Venue Longitude', 
                  #'Venue Category']
                    
    return(nearby_venues)

### Call FourSquare Get Function

In [16]:
denver_breweries = getNearbyVenues(names=den_df['Neighborhood'], 
                                 latitudes=den_df['Latitude'], 
                                 longitudes=den_df['Longitude']
                                )

Alder-Three Sisters Park
Allendale
Alta Vista and Memorial Park
Apel Bacher Park, Koch Sub and Coulehan Grange
Applewood, Echo Hill, Rolling Hills and Meadows
Arvada Center
Athmar Park
Auraria
Aurora Highlands and Side Creek
Baker
Barnum
Barnum West
Barr Lake State Park
Bear Valley
Belcaro
Bell Park
Benedict Park, Central Brighton
Berkeley
Berrian Mountain
Big Dry Creek
Boulder Municipal Airport
Bowels Grove Park
CU East Campus
CU Main Campus
Capitol Hill
Castle Pines North
Castle Pines Villige, Metes and Bounds
Castlewood Ranch
Centennial Airport
Central Business
Central Commerce City
Central East Aurora
Central North Broomfield
Central Northglenn
Central West Broomfield
Central south Littleton
Chaffee Park
Chambers Heights and Laredo Highline
Charles Whitlock Rec Park and Morse Park
Chautauqua, Table Mesa, and Devil's Thumb
Cheesman Park
Cherry Creek
Cherry Creek Reservoir
Cherry Hills Village Area
City Hall
City Park
City Park West
Civic Center
Clark Centennial Park
Clayton
Clement 

### Check shape and header information of new dataframe

In [17]:
#new df parameters
print(denver_breweries.shape)
denver_breweries.head()

(135, 6)


,0,1,2,3,4,5
0,Alder-Three Sisters Park,39.74001,-104.99202,Iron Hill Brewery Nation,39.741615,-104.997566
1,"Apel Bacher Park, Koch Sub and Coulehan Grange",39.74001,-104.99202,Iron Hill Brewery Nation,39.741615,-104.997566
2,"Applewood, Echo Hill, Rolling Hills and Meadows",39.74001,-104.99202,Iron Hill Brewery Nation,39.741615,-104.997566
3,Athmar Park,39.70396,-105.01039,Chain Reaction Brewery,39.699577,-105.001335
4,Auraria,39.74575,-105.00997,Breckenridge Brewery Mountain House,39.748078,-105.006897


### Name Columns

In [18]:
denver_breweries.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
                  

print('frame size',denver_breweries.shape)
print('data type', denver_breweries.dtypes)
denver_breweries.head()

frame size (135, 6)
data type Neighborhood               object
Neighborhood Latitude     float64
Neighborhood Longitude    float64
Venue                      object
Venue Latitude            float64
Venue Longitude           float64
dtype: object


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,Alder-Three Sisters Park,39.74001,-104.99202,Iron Hill Brewery Nation,39.741615,-104.997566
1,"Apel Bacher Park, Koch Sub and Coulehan Grange",39.74001,-104.99202,Iron Hill Brewery Nation,39.741615,-104.997566
2,"Applewood, Echo Hill, Rolling Hills and Meadows",39.74001,-104.99202,Iron Hill Brewery Nation,39.741615,-104.997566
3,Athmar Park,39.70396,-105.01039,Chain Reaction Brewery,39.699577,-105.001335
4,Auraria,39.74575,-105.00997,Breckenridge Brewery Mountain House,39.748078,-105.006897


### Backup new data frame to local disk

In [19]:
denver_breweries.to_csv('denver_breweries.csv')

### Group breweries by neighborhood

In [20]:
denver_breweries.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Neighborhood,,,,,
Alder-Three Sisters Park,1,1,1,1,1
"Apel Bacher Park, Koch Sub and Coulehan Grange",1,1,1,1,1
"Applewood, Echo Hill, Rolling Hills and Meadows",1,1,1,1,1
Athmar Park,1,1,1,1,1
Auraria,3,3,3,3,3
Baker,3,3,3,3,3
Bell Park,2,2,2,2,2
Berkeley,2,2,2,2,2
Bowels Grove Park,2,2,2,2,2


### One Hot Coding on Venue

In [21]:
den_onehot = pd.get_dummies(denver_breweries[['Venue']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
den_onehot['Neighborhood'] = denver_breweries['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [den_onehot.columns[-1]] + list(den_onehot.columns[:-1])
den_onehot = den_onehot[fixed_columns]

den_onehot.head()

,Neighborhood,Birch Street Brewery,Black Shirt Brewery,Black Sky Brewery,Blue Moon Brewery at The Sandlot,Boggy Draw Brewery,Breckenridge Brewery Mountain House,Brewery Bar,Brewery Bar III,Briar Common,...,Strange Craft Beer Company,The Brewery,The Grateful Gnome Sandwich Shoppe & Brewery,The Short Brewery,Vine Street Pub & Brewery,West Flanders Brewing Company,Wild Mountain Smokehouse & Brewery,Wit’s End Brewery,Wynkoop Brewing Co.,next day brewery
0,Alder-Three Sisters Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Apel Bacher Park, Koch Sub and Coulehan Grange",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Applewood, Echo Hill, Rolling Hills and Meadows",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Athmar Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Auraria,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group by Neighbordhood and mean frequency

In [22]:
den_grouped = den_onehot.groupby('Neighborhood').mean().reset_index()
den_grouped

,Neighborhood,Birch Street Brewery,Black Shirt Brewery,Black Sky Brewery,Blue Moon Brewery at The Sandlot,Boggy Draw Brewery,Breckenridge Brewery Mountain House,Brewery Bar,Brewery Bar III,Briar Common,...,Strange Craft Beer Company,The Brewery,The Grateful Gnome Sandwich Shoppe & Brewery,The Short Brewery,Vine Street Pub & Brewery,West Flanders Brewing Company,Wild Mountain Smokehouse & Brewery,Wit’s End Brewery,Wynkoop Brewing Co.,next day brewery
0,Alder-Three Sisters Park,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
1,"Apel Bacher Park, Koch Sub and Coulehan Grange",0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Applewood, Echo Hill, Rolling Hills and Meadows",0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,Athmar Park,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
4,Auraria,0.000000,0.0,0.0,0.00,0.0,0.333333,0.000000,0.0,0.333333,...,0.333333,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
5,Baker,0.000000,0.0,0.0,0.00,0.0,0.000000,0.333333,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
6,Bell Park,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
7,Berkeley,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.5,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
8,Bowels Grove Park,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
9,Capitol Hill,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


### Define Sort Function

In [23]:
#Function to sort venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Get top ten venues

In [24]:
#Top 10 Venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = den_grouped['Neighborhood']

for ind in np.arange(den_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(den_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alder-Three Sisters Park,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
1,"Apel Bacher Park, Koch Sub and Coulehan Grange",Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
2,"Applewood, Echo Hill, Rolling Hills and Meadows",Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
3,Athmar Park,Chain Reaction Brewery,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
4,Auraria,Breckenridge Brewery Mountain House,Strange Craft Beer Company,Briar Common,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.


### Wrangle and Normalize Denver Data Frame

In [25]:
brew_count = denver_breweries.groupby('Neighborhood').count()
brew_count.dtypes

Neighborhood Latitude     int64
Neighborhood Longitude    int64
Venue                     int64
Venue Latitude            int64
Venue Longitude           int64
dtype: object

In [26]:
df_merged = pd.merge(brew_count,den_df, on='Neighborhood')
df_merged.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Population',
       'Low Income', 'Medium Income', 'High Income', 'Latitude', 'Longitude'],
      dtype='object')

In [27]:
df_data = df_merged[['Population','Low Income', 'Medium Income', 'High Income','Venue']]
print('data types: ', df_data.dtypes)
df_data.head()

data types:  Population       int64
Low Income       int64
Medium Income    int64
High Income      int64
Venue            int64
dtype: object


,Population,Low Income,Medium Income,High Income,Venue
0,381,544,614,452,1
1,467,600,359,65,1
2,433,456,549,531,1
3,1840,1900,956,269,1
4,39,20,4,15,3


In [28]:
normalized_df=(df_data-df_data.min())/(df_data.max()-df_data.min())
normalized_df.head()

,Population,Low Income,Medium Income,High Income,Venue
0,0.044101,0.066254,0.127810,0.111138,0.0
1,0.055190,0.073334,0.074729,0.015982,0.0
2,0.050806,0.055127,0.114280,0.130563,0.0
3,0.232237,0.237704,0.199001,0.066142,0.0
4,0.000000,0.000000,0.000833,0.003688,0.5


## K-Means Analysis

In [29]:
## K-Means Analysis ##
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(normalized_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 4, 4, 1, 3, 1, 2], dtype=int32)

In [30]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = df_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
denver_merged = denver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

denver_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Population,Low Income,Medium Income,High Income,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alder-Three Sisters Park,1,1,1,1,1,381,544,614,452,...,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
1,"Apel Bacher Park, Koch Sub and Coulehan Grange",1,1,1,1,1,467,600,359,65,...,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
2,"Applewood, Echo Hill, Rolling Hills and Meadows",1,1,1,1,1,433,456,549,531,...,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
3,Athmar Park,1,1,1,1,1,1840,1900,956,269,...,Chain Reaction Brewery,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
4,Auraria,3,3,3,3,3,39,20,4,15,...,Breckenridge Brewery Mountain House,Strange Craft Beer Company,Briar Common,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.


### Clean up dataframe for plotting

In [ ]:
#toronto_merged = toronto_merged.dropna()
#toronto_merged = toronto_merged.astype({'Cluster Labels':'int32'})
#toronto_merged.head()

### Map Clusters

In [31]:
## Map Our New Clusters ###
# create map
map_clusters = folium.Map(location=[39.7392364, -104.9848623], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Neighborhood'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Save Map

In [32]:
map_clusters.save('map_clusters.html')

## Explore Clusters
### Cluster 1

In [33]:
### Cluster 1 ###
denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Population,Low Income,Medium Income,High Income,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,1,1,5182,6662,3881,1677,39.650391,-105.021878,0,Boggy Draw Brewery,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
29,1,1,3368,2260,4071,2589,39.740010,-104.992020,0,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
33,1,1,3437,2675,4300,2461,39.740010,-104.992020,0,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
40,1,1,2366,3029,3282,4067,39.760114,-105.009369,0,Prost Brewing,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
43,1,1,3710,2467,2409,2789,39.740010,-104.992020,0,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
56,1,1,3001,4813,2153,1549,39.760114,-105.009369,0,Prost Brewing,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
58,2,2,4615,3704,2265,1389,39.719330,-104.979980,0,Counter Culture Brewery + Grille,Black Shirt Brewery,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
61,1,1,4470,5115,2977,987,39.808880,-104.978080,0,River North Brewery,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company


### Cluster 2

In [35]:
### Cluster 2 ###
denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Population,Low Income,Medium Income,High Income,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,1,381,544,614,452,39.740010,-104.992020,1,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
1,1,1,467,600,359,65,39.740010,-104.992020,1,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
2,1,1,433,456,549,531,39.740010,-104.992020,1,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
3,1,1,1840,1900,956,269,39.703960,-105.010390,1,Chain Reaction Brewery,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
6,2,2,653,881,818,971,39.875605,-104.991202,1,Hops Restaurant,Mangy Dog Home Brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
8,2,2,621,550,737,524,39.616930,-105.028040,1,Jackass Hill Brewing Co.,Saint Patrick’s Brewery,Black Shirt Brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
11,2,2,1346,1182,533,797,39.743670,-104.990670,1,Jagged Mountain Brewery,Iron Hill Brewery Nation,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
15,1,1,1337,941,488,402,39.743790,-104.948250,1,Vine Street Pub & Brewery,next day brewery,Cooper Kettle Brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
16,2,2,1927,1601,563,502,39.745450,-104.967200,1,Vine Street Pub & Brewery,Gaijin 24886 Brewery,next day brewery,Cooper Kettle Brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Freshwerks Brewery
19,1,1,187,404,328,300,39.553710,-104.876680,1,Brewery Bar III,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery


### Cluster 3

In [36]:
### Cluster 3 ###
denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Population,Low Income,Medium Income,High Income,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,2,2,7794,7192,2960,946,39.733700,-104.979290,2,Counter Culture Brewery + Grille,Lowdown Brewery+Kitchen,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
12,4,4,4015,1607,3034,1323,39.756989,-105.011058,2,Wynkoop Brewing Co.,Breckenridge Brewery Mountain House,Briar Common,Prost Brewing,Crazy Mountain Brewery Taproom & Beer Garden,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.
18,4,4,5246,7929,4804,1649,39.752550,-105.008770,2,Wynkoop Brewing Co.,Breckenridge Brewery Mountain House,Briar Common,Prost Brewing,Crazy Mountain Brewery Taproom & Beer Garden,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.
25,5,5,6694,3532,2387,2044,39.759200,-104.987600,2,Jagged Mountain Brewery,Blue Moon Brewery at The Sandlot,Liberati Restaurant & Brewery,Great Divide Brewing Co.,Spangalang Brewery,next day brewery,Counter Culture Brewery + Grille,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery


### Cluster 4

In [37]:
### Cluster 4 ###
denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Population,Low Income,Medium Income,High Income,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,2,2,1970,1753,1321,1264,39.776660,-105.039840,3,The Grateful Gnome Sandwich Shoppe & Brewery,Empourium Brewing Company,next day brewery,Cooper Kettle Brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
10,1,1,1127,1017,1219,3509,39.875294,-105.009828,3,Mangy Dog Home Brewery,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
13,1,1,1039,1557,800,1908,39.717820,-104.948830,3,Hops Grill & Brewery,next day brewery,Iron Hill Brewery Nation,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
14,1,1,1717,1804,1702,2193,39.717820,-104.948830,3,Hops Grill & Brewery,next day brewery,Iron Hill Brewery Nation,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
21,2,2,3982,3020,1407,1466,39.733590,-104.950010,3,Birch Street Brewery,Freshwerks Brewery,Iron Hill Brewery Nation,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Empourium Brewing Company
30,1,1,2892,1694,1603,1472,39.764420,-105.011980,3,Prost Brewing,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
41,1,1,1618,1482,1111,1386,39.722510,-104.891540,3,Days Off Brewery,next day brewery,Iron Hill Brewery Nation,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
44,1,1,1123,1699,1643,960,39.740160,-104.990060,3,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery,Empourium Brewing Company
46,1,1,1209,1350,1515,1727,39.760114,-105.009369,3,Prost Brewing,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
48,2,2,2536,3188,1875,832,39.616900,-105.014520,3,Jackass Hill Brewing Co.,Saint Patrick’s Brewery,Black Shirt Brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery


### Cluster 5

In [38]:
### Cluster 5 ###
denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Population,Low Income,Medium Income,High Income,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,3,3,39,20,4,15,39.745750,-105.009970,4,Breckenridge Brewery Mountain House,Strange Craft Beer Company,Briar Common,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.
5,3,3,2042,1379,987,553,39.711170,-104.992090,4,Heavenly Daze Brewery,Brewery Bar,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery,Freshwerks Brewery
17,3,3,516,487,436,376,39.735260,-104.990580,4,Counter Culture Brewery + Grille,Lowdown Brewery+Kitchen,Iron Hill Brewery Nation,next day brewery,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
23,4,4,554,836,959,631,39.758354,-104.982106,4,Blue Moon Brewery at The Sandlot,Liberati Restaurant & Brewery,Great Divide Brewing Co.,Spangalang Brewery,next day brewery,Counter Culture Brewery + Grille,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Gaijin 24886 Brewery
28,3,3,1900,2240,1082,806,39.732710,-104.930420,4,next day brewery,Freshwerks Brewery,Birch Street Brewery,Wild Mountain Smokehouse & Brewery,Cooper Kettle Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
34,3,3,1640,994,411,339,39.751210,-105.021350,4,Little Machine Beer,The Brewery,Briar Common,next day brewery,Counter Culture Brewery + Grille,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
36,3,3,2412,1979,1796,728,39.747528,-104.978300,4,Liberati Restaurant & Brewery,Great Divide Brewing Co.,Spangalang Brewery,next day brewery,Cooper Kettle Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Gaijin 24886 Brewery,Freshwerks Brewery
37,5,5,1612,1757,767,226,39.728960,-105.002160,4,Black Sky Brewery,Heavenly Daze Brewery,Brewery Bar,Crazy Mountain Brewery Taproom & Beer Garden,next day brewery,Counter Culture Brewery + Grille,Hops Grill & Brewery,Hogshead Brewery,Grossen Bart Brewery,Great Divide Brewing Co.
42,4,4,666,583,533,859,40.019950,-105.286080,4,Redfish Fishouse and Brewery,West Flanders Brewing Company,Mountain Sun Pub & Brewery,Chicago Pizza and Brewery,next day brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery,Great Divide Brewing Co.,Gaijin 24886 Brewery
62,5,5,2218,1302,1128,1516,39.753490,-104.998880,4,Wynkoop Brewing Co.,Jagged Mountain Brewery,Blue Moon Brewery at The Sandlot,Breckenridge Brewery Mountain House,Great Divide Brewing Co.,Crazy Mountain Brewery Taproom & Beer Garden,Hops Grill & Brewery,Hogshead Brewery,Heavenly Daze Brewery,Grossen Bart Brewery
